In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
dubai_apartments = pd.read_json('/content/drive/MyDrive/Projector_traning/dubaiApp.json')
dubai_apartments.head()

,id,title,displayAddress,bedrooms,bathrooms,addedOn,verified,priceDuration,propertyType,price,...,coordinates,type,sizeMin,furnishing,features,description,sizeSqft,sizeSqm,priceUSD,pricePerSqmUSD
0,11272474,Spacious Studio l Pool View l High ROI,"Lincoln Park B, Lincoln Park, Arjan, Dubai",0,1,2024-03-20T06:06:46Z,True,sell,Apartment,425000,...,"{'latitude': 25.061007, 'longitude': 55.238113}",Residential for Sale,478 sqft,YES,"[Central A/C, Balcony, Shared Pool, Security, ...",Offering you a spacious Studio apartment avail...,478,44.407284,115740.740741,2606.345886
1,11382004,Good Deal | Spacious | High Floor | Must See!,"Al Andalus Tower C, Al Andalus, Jumeirah Golf ...",1,1,2024-04-12T09:42:46Z,True,sell,Apartment,925000,...,"{'latitude': 25.030499, 'longitude': 55.206486}",Residential for Sale,777 sqft,NO,"[Central A/C, Balcony, Children's Play Area]",Top Class Real Estate gladly offers 1 Bedroom ...,777,72.185061,251906.318083,3489.729225
2,11354794,Studio | Q2 26 | 60/40 PP | 2 YRS Post,"Helvetia Residences, Jumeirah Village Circle, ...",0,1,2024-04-04T06:50:53Z,False,sell,Apartment,690000,...,"{'latitude': 25.04747, 'longitude': 55.205948}",Residential for Sale,445 sqft,PARTLY,[Balcony],A studio apartment in Helvetia Residences by D...,445,41.341509,187908.496732,4545.274289
3,11354795,Studio | Q2 26 | 60/40 PP | 2 YRS Post,"Helvetia Residences, Jumeirah Village Circle, ...",0,1,2024-04-04T06:50:54Z,False,sell,Apartment,692000,...,"{'latitude': 25.04747, 'longitude': 55.205948}",Residential for Sale,445 sqft,PARTLY,[Balcony],A studio apartment in Helvetia Residences by D...,445,41.341509,188453.159041,4558.448998
4,11191211,Prime Location | Pool View | Modern Furnishing,"Jewelz by Danube, Arjan, Dubai",0,1,2024-03-06T07:28:28Z,True,sell,Apartment,675000,...,"{'latitude': 25.064808, 'longitude': 55.24055}",Residential for Sale,422 sqft,YES,"[Central A/C, Shared Pool, Covered Parking, Bu...",Azco Real Estate is thrilled to offer a stunni...,422,39.204757,183823.529412,4688.806802


**ADDRESS:**

In [3]:
dubai_apartments['displayAddress'] = dubai_apartments['displayAddress'].str.lower()
split_addresses = dubai_apartments['displayAddress'].str.split(', ')

# Flatten the list of lists and count the frequency of each substring
address_freq = pd.Series([item for sublist in split_addresses for item in sublist]).value_counts()

print(address_freq.shape)
pd.set_option('display.max_rows', 100)
print(address_freq[0:100])
pd.reset_option('display.max_rows')

(2730,)
dubai                                74845
jumeirah village circle               9989
business bay                          4970
jumeirah village triangle             4525
mohammed bin rashid city              4508
dubai investment park                 4158
downtown dubai                        3742
arjan                                 3499
dubai marina                          3059
verdana 2                             2251
al furjan                             2199
dubai creek harbour (the lagoons)     2198
palm jumeirah                         2146
majan                                 2113
dubai harbour                         1989
sobha hartland                        1838
red square tower                      1778
dubai production city (impz)          1715
jumeirah lake towers                  1595
emaar beachfront                      1550
dubai sports city                     1527
dubai hills estate                    1501
verdana                               1396
jeb

In [4]:
split_regions = split_addresses.apply(lambda x: x[1:] if len(x) > 1 else [])
regions = set([item for sublist in split_regions for item in sublist])
print(len(regions))
print(regions)

432
{'al majara', 'kingdom of sheba', 'wasl1', 'cbd (central business district)', 'le presidium', 'widcombe house', 'golf panorama', 'the address residences dubai opera', 'dubai waterfront', 'hub-golf towers', 'mudon', 'liwan', 'remraam', 'sparkle towers', 'jumeirah 2', 'oceanic', 'al sidir', 'celestia', 'dubai south (dubai world central)', 'marina residence', '15 northside', 'dubai hills estate', 'tecom two towers', 'bluewaters residences', 'mag 5', 'amwaj', 'kensington waters', 'lago vista', 'sadaf', 'terrace apartments', 'barton house', 'royal residence', 'green lake towers', 'murjan', 'address harbour point', 'al quoz 4', 'the arena apartments', 'marina diamonds', 'safeer towers', 'creek gate', 'collective 2.0', 'reehan', 'uptown mirdif', 'park ridge', 'greece cluster', 'turia', 'dubai studio city', 'trident marinascape', 'madinat badr', 'peninsula', 'harbour residences', 'the views', 'indigo towers', 'italy cluster', 'starz by danube', 'naia golf terrace at akoya', 'park towers', 

In [5]:
for index, row in dubai_apartments.iterrows():
    addresses = row['displayAddress'].split(', ')
    if addresses[0] in regions:
      dubai_apartments.loc[index, 'region'] = next((adr for adr in addresses if address_freq[adr] >= 200), 'dubai')
      dubai_apartments.loc[index, 'tower'] = 'unknown'
    else:
      dubai_apartments.loc[index, 'region'] = next((adr for adr in addresses[1:] if address_freq[adr] >= 200), 'dubai')
      dubai_apartments.loc[index, 'tower'] = addresses[0] if address_freq[addresses[0]] >= 10 else 'unknown'

In [6]:
regon_counts = dubai_apartments['region'].value_counts(ascending=False)
regon_counts[regon_counts >= 200]

region
jumeirah village circle      9989
jumeirah village triangle    4525
dubai investment park        4158
business bay                 3845
dubai                        3739
                             ... 
damac hills 2                 214
dubai silicon oasis           211
al habtoor city               209
forte                         203
design quarter                203
Name: count, Length: 61, dtype: int64

In [10]:
tower_counts = dubai_apartments['tower'].value_counts(ascending=False)
tower_counts[tower_counts >= 10]

tower
unknown                     7328
verdana 2                   2251
red square tower            1778
verdana                     1396
eleve by deyaar             1035
                            ... 
elite sports residence 2      10
kensington waters b           10
al thayyal 3                  10
eaton place                   10
venetian                      10
Name: count, Length: 1192, dtype: int64

In [ ]:
#dubai_apartments.drop(columns=['min', 'max', 'quantile_10', 'quantile_20', 'quantile_80', 'quantile_90'], inplace=True)

In [11]:
tower_quantiles = dubai_apartments.groupby('tower')['pricePerSqmUSD'].quantile([0.2, 0.8]).unstack().reset_index()
tower_quantiles.columns = ['tower', 'quantile_20', 'quantile_80']

# Step 2: Merge min and max prices back into the original DataFrame
dubai_apartments = pd.merge(dubai_apartments, tower_quantiles, on='tower')

# Step 3: Calculate deviation of price from min and max in percent
dubai_apartments['deviationFromMinPrice'] = ((dubai_apartments['quantile_20'] - dubai_apartments['pricePerSqmUSD']) / dubai_apartments['pricePerSqmUSD']) * 100
dubai_apartments['deviationFromMaxPrice'] = ((dubai_apartments['quantile_80'] - dubai_apartments['pricePerSqmUSD']) / dubai_apartments['pricePerSqmUSD']) * 100

# Step 4: Calculate average deviation across all apartments and regions
average_deviation_min = dubai_apartments[dubai_apartments['tower'] != 'unknown']['deviationFromMinPrice'].mean()
average_deviation_max = dubai_apartments[dubai_apartments['tower'] != 'unknown']['deviationFromMaxPrice'].mean()

print("Average deviations:", f'{int(average_deviation_min)}%', f'{int(average_deviation_max)}%')

Average deviations: -9% 16%


**FLOOR:**

In [12]:
import re

dubai_apartments['title'] = dubai_apartments['title'].str.lower().str.replace(r'\W|_', ' ')
dubai_apartments['titleFloor'] = dubai_apartments['title'].str.extract(r'(\b\w+\b\s+floor\b)', flags=re.IGNORECASE)
dubai_apartments['description'] = dubai_apartments['description'].str.lower().replace(r'\W|_', ' ')
dubai_apartments['descriptionFloor'] = dubai_apartments['description'].str.extract(r'(\b\w+\b\s+floor\b)', flags=re.IGNORECASE)
dubai_apartments['floor'] = dubai_apartments.apply(lambda x: x['descriptionFloor'] if pd.isna(x['titleFloor']) else x['titleFloor'], axis=1)
pd.set_option('display.max_rows', 350)
print(dubai_apartments['floor'].value_counts(dropna=False)[:350])
pd.reset_option('display.max_rows')

floor
NaN                    53631
high floor              9361
mid floor               1692
ground floor            1424
low floor               1383
19th floor               505
higher floor             399
middle floor             348
full floor               335
top floor                327
open floor               278
with floor               274
and floor                216
the floor                166
lower floor              148
spacious floor           147
10th floor               124
boasts floor             118
first floor              116
highest floor            110
large floor              108
3rd floor                 92
expansive floor           88
to floor                  88
thoughtful floor          85
featuring floor           65
through floor             65
every floor               65
6th floor                 64
designed floor            64
per floor                 58
crafted floor             58
4th floor                 53
42nd floor                52
15th flo

In [13]:
dubai_apartments['floor'] = dubai_apartments['floor'].apply(lambda x: x if x not in {'high floor', 'higher floor', 'highest floor', 'top floor', '19th floor'} else 'high floor')
dubai_apartments['floor'] = dubai_apartments['floor'].apply(lambda x: x if x not in {'mid floor', 'middle floor', '10th floor'} else 'mid floor')
dubai_apartments['floor'] = dubai_apartments['floor'].apply(lambda x: x if x not in {'low floor', 'lower floor'} else 'low floor')
dubai_apartments['floor'] = dubai_apartments['floor'].apply(lambda x: x if x not in {'ground floor', 'first floor'} else 'ground floor')

dubai_apartments['floor'] = dubai_apartments['floor'].apply(lambda x: x if x in {'high floor', 'mid floor', 'low floor', 'ground floor'} else 'unknown')

print(dubai_apartments['floor'].value_counts(dropna=False))

floor
unknown         58908
high floor      10702
mid floor        2164
ground floor     1540
low floor        1531
Name: count, dtype: int64


**ATLANTIS**

In [14]:
dubai_apartments[dubai_apartments['tower'].apply(lambda x: x.startswith('atlantis'))][['displayAddress', 'tower', 'region', 'sizeSqm', 'priceUSD', 'pricePerSqmUSD', 'floor']]

,displayAddress,tower,region,sizeSqm,priceUSD,pricePerSqmUSD,floor
71727,"atlantis the royal residences, palm jumeirah, ...",atlantis the royal residences,palm jumeirah,583.519138,1.879085e+07,32202.627907,unknown
71728,"atlantis the royal residences, palm jumeirah, ...",atlantis the royal residences,palm jumeirah,190.356745,5.174292e+06,27182.076867,unknown
71729,"atlantis the royal residences, palm jumeirah, ...",atlantis the royal residences,palm jumeirah,316.796730,1.361656e+07,42982.002185,high floor
71730,"atlantis the royal residences, palm jumeirah, ...",atlantis the royal residences,palm jumeirah,579.617243,2.450980e+07,42286.188397,high floor
71731,"atlantis the royal residences, palm jumeirah, ...",atlantis the royal residences,palm jumeirah,190.356745,5.255991e+06,27611.269007,unknown
71732,"atlantis the royal residences, palm jumeirah, ...",atlantis the royal residences,palm jumeirah,341.044221,1.089325e+07,31940.861335,unknown
71733,"atlantis the royal residences, palm jumeirah, ...",atlantis the royal residences,palm jumeirah,324.879227,8.986928e+06,27662.366062,unknown
71734,"atlantis the royal residences, palm jumeirah, ...",atlantis the royal residences,palm jumeirah,319.212189,8.959695e+06,28068.148097,unknown
71735,"atlantis the royal residences, palm jumeirah, ...",atlantis the royal residences,palm jumeirah,253.901895,5.991285e+06,23596.851840,unknown
71736,"atlantis the royal residences, palm jumeirah, ...",atlantis the royal residences,palm jumeirah,177.722036,5.310458e+06,29880.692476,low floor


**ENCODING:**

In [15]:
from sklearn.preprocessing import LabelEncoder
regionEncoder = LabelEncoder()
dubai_apartments['region'] = regionEncoder.fit_transform(dubai_apartments['region'])
towerEncoder = LabelEncoder()
dubai_apartments['tower'] = towerEncoder.fit_transform(dubai_apartments['tower'])
floorEncoder = LabelEncoder()
dubai_apartments['floor'] = floorEncoder.fit_transform(dubai_apartments['floor'])

**COORDINATES:**

In [16]:
type(dubai_apartments.loc[0, 'coordinates'])

dict

In [17]:
dubai_apartments['coordinatesStr'] = dubai_apartments['coordinates'].apply(lambda x: f"{x['latitude']}#{x['longitude']}")
dubai_apartments['coordinatesStr'].value_counts()

coordinatesStr
24.989403#55.18063              2251
25.040577#55.183754             1778
24.991396#55.179836             1396
24.978566999999998#55.092262    1031
25.097109#55.314262              918
                                ... 
24.984732#55.388977                1
25.040035#55.21979                 1
25.107576#55.142998                1
25.014614#55.240093                1
25.09833#55.170116                 1
Name: count, Length: 2983, dtype: int64

In [18]:
dubai_apartments['latitude'] = dubai_apartments['coordinates'].apply(lambda x: x['latitude'])
dubai_apartments['longitude'] = dubai_apartments['coordinates'].apply(lambda x: x['longitude'])

dubai_center = { 'latitude': 25.118349833941004, 'longitude': 55.2006187248306 }
dubai_apartments['latitudeNorm'] = dubai_apartments['latitude'] - dubai_center['latitude']
dubai_apartments['longitudeNorm'] = dubai_apartments['longitude'] - dubai_center['longitude']

In [19]:
from pyproj import Proj, transform, Transformer

"""proj_latlon = Proj(proj='latlong', datum='WGS84')
proj_utm = Proj(proj='utm', zone=40, datum='WGS84')
dubai_apartments['x'], dubai_apartments['y'] = transform(proj_latlon, proj_utm, dubai_apartments['longitude'], dubai_apartments['latitude'])
dubai_center_x, dubai_center_y = transform(proj_latlon, proj_utm, dubai_center['longitude'], dubai_center['latitude'])"""

transformer = Transformer.from_crs("epsg:4326", "epsg:32640", always_xy=True)
dubai_apartments['x'], dubai_apartments['y'] = transformer.transform(dubai_apartments['longitude'], dubai_apartments['latitude'])
dubai_center_x, dubai_center_y = transformer.transform(dubai_center['longitude'], dubai_center['latitude'])

dubai_apartments['xNorm'] = dubai_apartments['x'] - dubai_center_x
dubai_apartments['yNorm'] = dubai_apartments['y'] - dubai_center_y

In [20]:
from geopy.distance import geodesic

dubai_airport = (25.256829022536586, 55.36428560948998)
dubai_apartments['dubaiAirportDistance'] = dubai_apartments.apply(lambda x: geodesic(dubai_airport, (x['latitude'], x['longitude'])).meters, axis=1)

In [21]:
burj_khalifa = (25.19739107957529, 55.27437637356584)
dubai_apartments['burjKhalifaDistance'] = dubai_apartments.apply(lambda x: geodesic(burj_khalifa, (x['latitude'], x['longitude'])).meters, axis=1)

In [22]:
burj_al_arab = (25.141273184617518, 55.18524651990685)
dubai_apartments['burjAlArabDistance'] = dubai_apartments.apply(lambda x: geodesic(burj_al_arab, (x['latitude'], x['longitude'])).meters, axis=1)

In [23]:
dubai_marina_center = (25.077735597715016, 55.138607180066)
dubai_apartments['dubaiMarinaDistance'] = dubai_apartments.apply(lambda x: geodesic(dubai_marina_center, (x['latitude'], x['longitude'])).meters, axis=1)

In [24]:
palm_jumeirah_center = (25.120569077809723, 55.13019494003337)
dubai_apartments['palmJumeirahDistance'] = dubai_apartments.apply(lambda x: geodesic(palm_jumeirah_center, (x['latitude'], x['longitude'])).meters, axis=1)

In [25]:
mall_of_emirates = (25.11834011969971, 55.200640182502866)
dubai_apartments['mallOfEmiratesDistance'] = dubai_apartments.apply(lambda x: geodesic(mall_of_emirates, (x['latitude'], x['longitude'])).meters, axis=1)

In [26]:
dubai_beaches = [
    (25.081463719021993, 55.13397935950754), # JBR beach
    (25.117865396881008, 55.167346700406185), # Sufouh beach
    (25.1421628373442, 55.18827741327512), # Jumeirah beach
    (25.164773770706013, 55.207503237278694), # Kite beach
    (25.186542020197347, 55.22434001498659), # Night beach
    (25.227714773367836, 55.25663605123227), # Le mer beach
]

dubai_apartments['nearestBeachDistance'] = dubai_apartments.apply(
    lambda x: min([
          geodesic(beach, (x['latitude'], x['longitude'])).meters for beach in dubai_beaches
        ]),
    axis=1
)

In [27]:
"""dubai_metro_stations = [
    (24.97759, 55.09108), # UAE Exchange
    (25.00110, 55.09575), # Danube
    (25.02641, 55.1012625), # Energy
    (25.046778, 55.117528), # Ibn Battuta
    (24.97759, 55.09108), # UAE Exchange
    (24.97759, 55.09108), # UAE Exchange
    (24.97759, 55.09108), # UAE Exchange
    (24.97759, 55.09108), # UAE Exchange
]"""

dubai_metro_stations = [
    (25.255282851001148, 55.33083550844358), # City Centre Deira Metro Station1
    (24.977805239601814, 55.09162156795771), # UAE Exchange Metro Station
    (25.07135885322516, 55.13990180050174), # DMCC Metro Station
    (25.266607745744174, 55.3151169391322), # Union Metro Station
    (25.201543691969903, 55.26970092986803), # Burj Khalifa/ Dubai Mall Metro Station
    (25.121451944598434, 55.20079060774886), # Mall of the Emirates Metro Station

    (25.046992864419682, 55.11815085261602), # Ibn Battuta Metro Station
    (25.268137040547742, 55.314089521432166), # Al Rigga Metro Station
    (25.191408930063123, 55.2612252965388), # Business Bay Metro Station
    (25.277941615216378, 55.36175052949552), # Stadium Metro Station
    (25.231249415044562, 55.323544488201705), # Healthcare City Metro Station
    (25.102210088302634, 55.17469042140428), # Dubai Internet City Metro Station

    (25.270312658982704, 55.37560805262427), # Dubai Airport Free Zone Metro Station
    (25.230489716430483, 55.392885397088826), # Rashidiya Metro Station
    (25.21915013238988, 55.33970904204091), # Creek Metro Station
    (25.00594574233904, 55.156279866106274), # Dubai Investment Park Metro Station
    (25.231133009270586, 55.323104631721524), # Dubai Healthcare City Metro Station
    (25.0890583019322, 55.159326414624786), # Al Khail Metro Station

    (25.25851954499461, 55.2976414485962), # Sharaf DG
    (25.081590337578774, 55.15381693969585), # Sobha Realty Metro Station
    (25.126492680967914, 55.20809259215247), # Equiti Metro Station
    (25.115328823333794, 55.19376302470839), # Mashreq Metro Station
    (25.16167753268869, 55.247696009645296), # Onpassive Metro Station
    (25.251049744786197, 55.314011024726035), # Oud Metha Metro Station

    (25.229445177401065, 55.343263475595336), # Al Jadaf
    (25.02141547863627, 55.19257942342074), # Jumeirah Golf Estates Metro Station
    (25.063556301007907, 55.148224537144834), # Jabal Ali Metro Station
    (24.965553766873793, 55.167793934100914), # Expo 2020 Metro Station
    (25.221898886786132, 55.29547159852185), # Emirates Towers Metro Station
    (25.245065632507632, 55.37948678442989), # Emirates Metro Station

    (25.236991515589207, 55.30275414894402), # Max Metro Station
    (25.248264997551477, 55.29949142941762), # Станция метро ADCB
    (25.247064064701032, 55.30935666139549), # Oud Metha Metro Station
]

dubai_apartments['nearestMetroStationDistance'] = dubai_apartments.apply(
    lambda x: min([
          geodesic(station, (x['latitude'], x['longitude'])).meters for station in dubai_metro_stations
        ]),
    axis=1
)

**FEATURES:**

In [28]:
type(dubai_apartments.loc[0, 'features'])

list

In [29]:
apartment_features = pd.Series([item for sublist in dubai_apartments['features'] for item in sublist]).unique()
apartment_features_freq = pd.Series([item for sublist in dubai_apartments['features'] for item in sublist]).value_counts()

print(len(apartment_features))
print(apartment_features)
print(apartment_features_freq)

In [30]:
#dummy_features = dubai_apartments['features'].str.join('|').str.get_dummies()
dummy_features = dubai_apartments['features'].apply(lambda x: '|'.join(x)).str.get_dummies()
dummy_features = dummy_features.astype(bool)
dubai_apartments = pd.concat([dubai_apartments, dummy_features], axis=1)
dummy_features

,Balcony,Barbecue Area,Built in Wardrobes,Central A/C,Children's Play Area,Children's Pool,Concierge,Covered Parking,Kitchen Appliances,Lobby in Building,...,Private Pool,Security,Shared Gym,Shared Pool,Shared Spa,Study,Vastu-compliant,View of Landmark,View of Water,Walk-in Closet
0,True,False,True,True,False,False,False,True,True,True,...,False,True,True,True,False,False,False,True,False,False
1,False,False,True,False,True,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
2,True,False,True,True,False,False,False,True,False,False,...,False,True,True,True,False,False,False,False,False,False
3,False,False,False,True,True,False,False,True,False,False,...,False,True,True,True,False,False,True,True,False,False
4,True,False,True,True,False,False,False,False,False,False,...,False,True,True,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74840,True,False,False,False,True,False,False,False,False,False,...,False,True,False,False,False,True,False,True,False,True
74841,True,False,False,False,True,False,False,False,False,False,...,False,True,False,False,False,True,False,True,False,True
74842,True,False,False,False,True,False,False,False,False,False,...,False,True,False,False,False,True,False,True,False,True
74843,True,False,False,False,True,False,False,False,False,False,...,False,True,False,False,False,True,False,True,False,True


In [31]:
dubai_apartments.to_json('/content/drive/MyDrive/Projector_traning/dubaiAppatrments.json')